#### importing data and files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime as dt
import statsmodels.api as sm
import scipy.stats as st
import re
import math
import collections
from matplotlib import rcParams
from matplotlib import gridspec
from pandas_profiling import ProfileReport
from sklearn import linear_model
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from collections import Counter
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
%matplotlib inline

# chargement des données
from google.colab import drive
drive.mount("/content/drive")

# dataframes
tr = pd.read_csv("/content/drive/MyDrive/OC/P4/P4_official_files/dataset_P4/transactions.csv")
pr = pd.read_csv("/content/drive/MyDrive/OC/P4/P4_official_files/dataset_P4/products.csv")
cl = pd.read_csv("/content/drive/MyDrive/OC/P4/P4_official_files/dataset_P4/customers.csv")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Part 1 -- Data cleaning

## Customers table

#### Checking file and dropping unecessary data

In [3]:
# quick overview of the file
cl.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [4]:
# checking data info
cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [5]:
# searching if there are null values
cl[cl.isnull().any(axis=1)]

,client_id,sex,birth


In [6]:
# detecting possible "anomalie"
cl.sort_values(by = "client_id", ascending=True).tail()

# ct_0 and ct_1 seems to be "test values"

,client_id,sex,birth
94,c_997,f,1994
2145,c_998,m,2001
7358,c_999,m,1964
2735,ct_0,f,2001
8494,ct_1,m,2001


In [7]:
# locating the "test" clients
print(f'THESE ARE THE "TEST" CUSTOMERS: \n\n{cl.loc[cl["client_id"].str.contains("ct", case=False)]}\n\nHence we have to drop it from the table...\n------------------------------------------\n')

# dropping them from the table
cl.drop([2735, 8494], inplace=True)
cl.loc[cl["client_id"].str.contains("ct", case=False)]

THESE ARE THE "TEST" CUSTOMERS: 

     client_id sex  birth
2735      ct_0   f   2001
8494      ct_1   m   2001

Hence we have to drop it from the table...
------------------------------------------



,client_id,sex,birth


In [8]:
# last overview
cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8621 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8621 non-null   object
 1   sex        8621 non-null   object
 2   birth      8621 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 269.4+ KB


#### Adding a new column for customer's Age

In [9]:
# we will drop the "birth" column as well, since we won't be needing it anymore
cl["age"] = 2022 - cl["birth"]
cl.drop(columns=["birth"], inplace=True)
cl.head()

,client_id,sex,age
0,c_4410,f,55
1,c_7839,f,47
2,c_1699,f,38
3,c_5961,f,60
4,c_5320,m,79


## Products table

#### Checking file and dropping unecessary data

In [10]:
# quick overview of the file
pr.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [11]:
# checking data info
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [12]:
# searching if there are null values
pr[pr.isnull().any(axis=1)]

,id_prod,price,categ


In [13]:
# detecting possible "anomalie"
pr.sort_values(by = "price", ascending=True).head()

# T_0 seems to be a "test value"

,id_prod,price,categ
731,T_0,-1.00,0
2355,0_202,0.62,0
2272,0_528,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0


In [14]:
# locating the "test" product
print(f'THIS IS THE "TEST" PRODUCT: \n\n{pr.loc[pr["id_prod"] == "T_0"]}\n\nThus we have to drop it from the table...\n------------------------------------------\n')

# dropping them from the table
pr.drop(pr[pr["id_prod"] == "T_0"].index, inplace=True)

# let's verify if well executed
print(f'Index 731 cannot be found,\nwe successfully dropped the unecessary data\n\n{pr.loc[pr["id_prod"] == "T_0"]}\n\n------------------------------------------\n')
pr.sort_values(by = "price", ascending=True).head()

THIS IS THE "TEST" PRODUCT: 

    id_prod  price  categ
731     T_0   -1.0      0

Thus we have to drop it from the table...
------------------------------------------

Index 731 cannot be found,
we successfully dropped the unecessary data

Empty DataFrame
Columns: [id_prod, price, categ]
Index: []

------------------------------------------



,id_prod,price,categ
2272,0_528,0.62,0
2355,0_202,0.62,0
370,0_120,0.66,0
1211,0_1844,0.77,0
1530,0_1620,0.80,0


In [15]:
# last overview
pr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3286 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3286 non-null   object 
 1   price    3286 non-null   float64
 2   categ    3286 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 102.7+ KB


## Transactions table

#### Checking file and dropping unecessary data

In [16]:
# quick overview of the file
tr.head()

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242


In [17]:
# checking data info
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


In [18]:
# searching if there are null values
tr[tr.isnull().any(axis=1)]

,id_prod,date,session_id,client_id


In [19]:
# detecting possible "anomalie"
tr.sort_values(by = "id_prod", ascending=False).head()

# T_0 seems to be a "test value"
# T_0 has client_id ct_0 and ct_1 who were mentioned as "tests values" previously

,id_prod,date,session_id,client_id
210294,T_0,test_2021-03-01 02:30:02.237432,s_0,ct_0
75176,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_1
203418,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0
259192,T_0,test_2021-03-01 02:30:02.237444,s_0,ct_0
234706,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_1


In [20]:
# locating the "test" product in Transactions
print(f'THERE ARE {len(tr.loc[tr["id_prod"] == "T_0"])} "TEST" VALUES FOUND IN TRANSACTIONS: \n\n{tr.loc[tr["id_prod"] == "T_0"]}\n\nThen, we have to drop them from the table...\n------------------------------------------\n')

# dropping them from the table
tr.drop(tr[tr["id_prod"] == "T_0"].index, inplace=True)
tr.loc[tr["id_prod"] == "T_0"]

THERE ARE 200 "TEST" VALUES FOUND IN TRANSACTIONS: 

       id_prod                             date session_id client_id
1431       T_0  test_2021-03-01 02:30:02.237420        s_0      ct_1
2365       T_0  test_2021-03-01 02:30:02.237446        s_0      ct_1
2895       T_0  test_2021-03-01 02:30:02.237414        s_0      ct_1
5955       T_0  test_2021-03-01 02:30:02.237441        s_0      ct_0
7283       T_0  test_2021-03-01 02:30:02.237434        s_0      ct_1
...        ...                              ...        ...       ...
332594     T_0  test_2021-03-01 02:30:02.237445        s_0      ct_0
332705     T_0  test_2021-03-01 02:30:02.237423        s_0      ct_1
332730     T_0  test_2021-03-01 02:30:02.237421        s_0      ct_1
333442     T_0  test_2021-03-01 02:30:02.237431        s_0      ct_1
335279     T_0  test_2021-03-01 02:30:02.237430        s_0      ct_0

[200 rows x 4 columns]

Then, we have to drop them from the table...
------------------------------------------



,id_prod,date,session_id,client_id


In [21]:
# last overview
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     336816 non-null  object
 1   date        336816 non-null  object
 2   session_id  336816 non-null  object
 3   client_id   336816 non-null  object
dtypes: object(4)
memory usage: 12.8+ MB


#### Fixing the date type

In [22]:
tr

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [23]:
tr["full_time"] = tr["date"]

In [24]:
# getting rid of microseconds:
def date_claire(date):
    return(date.split(".")[0])

tr["date"] = tr["date"].apply(date_claire)
tr.head(1)

,id_prod,date,session_id,client_id,full_time
0,0_1483,2021-04-10 18:37:28,s_18746,c_4450,2021-04-10 18:37:28.723910


In [25]:
# let's rename the "date" column to "date_and_time" then convert it to datetime:
tr.rename(columns={"date": "date_and_time"}, inplace=True)
tr["date_and_time"] = pd.to_datetime(tr["date_and_time"])
tr["full_time"] = pd.to_datetime(tr["full_time"])
tr.head(1)

,id_prod,date_and_time,session_id,client_id,full_time
0,0_1483,2021-04-10 18:37:28,s_18746,c_4450,2021-04-10 18:37:28.723910


In [26]:
tr.dtypes

id_prod                  object
date_and_time    datetime64[ns]
session_id               object
client_id                object
full_time        datetime64[ns]
dtype: object

In [27]:
# splitting the date into multiple categories:
# day
# day number
# month name
# month number
# year
# quarterly

tr.loc[:, "date_and_time"] = pd.to_datetime(tr.loc[:, "date_and_time"], errors="coerce")

tr["month"] = tr["date_and_time"].dt.strftime("%B")
tr["day"] = tr["date_and_time"].map(lambda d: d.day)
tr["year"] = tr["date_and_time"].map(lambda d: d.year)

tr["quarterly"] = [int((day-1)*4/31)+1 for day in tr["day"]]
tr["_monthnum"] = tr["date_and_time"].map(lambda d: d.month)
tr["_daynum"] = tr["date_and_time"].map(lambda d: d.weekday()+1)

tr["day_name"] = tr["_daynum"].map({1:str("Monday"), 2:str("Tuesday"), 3:str("Wednesday"), 4:str("Thursday"), 5:str("Friday"), 6:str("Saturday"), 7:str("Sunday")})

tr.head(2)

,id_prod,date_and_time,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name
0,0_1483,2021-04-10 18:37:28,s_18746,c_4450,2021-04-10 18:37:28.723910,April,10,2021,2,4,6,Saturday
1,2_226,2022-02-03 01:55:53,s_159142,c_277,2022-02-03 01:55:53.276402,February,3,2022,1,2,4,Thursday


In [28]:
# the date_and_time to datetime conversion will let us apply get_date() and get_hour()
# this will give us separate columns for date and for hour
def get_date(dh):
    return(dh.date());
tr["date"] = tr["date_and_time"].apply(get_date)

def get_hour(dh):
    return(dh.time());
tr["hour"] = tr["date_and_time"].apply(get_hour)

# we won't be needing the date_and_time column anymore in this case, so we can drop it
tr.drop(columns="date_and_time", inplace=True)
tr.head(2)

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour
0,0_1483,s_18746,c_4450,2021-04-10 18:37:28.723910,April,10,2021,2,4,6,Saturday,2021-04-10,18:37:28
1,2_226,s_159142,c_277,2022-02-03 01:55:53.276402,February,3,2022,1,2,4,Thursday,2022-02-03,01:55:53


# Part 2 -- Data preparation 

### TP table -- Transactions and Products

In [29]:
# merging Transactions and Products first
tp = pd.merge(tr, pr, on="id_prod", how="left").copy(); tp.head(2)

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ
0,0_1483,s_18746,c_4450,2021-04-10 18:37:28.723910,April,10,2021,2,4,6,Saturday,2021-04-10,18:37:28,4.99,0.0
1,2_226,s_159142,c_277,2022-02-03 01:55:53.276402,February,3,2022,1,2,4,Thursday,2022-02-03,01:55:53,65.75,2.0


In [30]:
# summation of total turnover
tp["price"].sum()

5796577.770000001

#### Checking file

In [31]:
# checking for any possible missing values
tp[tp.isnull().any(axis=1)]

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ
6231,0_2245,s_49705,c_1533,2021-06-17 03:03:12.668129,June,17,2021,3,6,4,Thursday,2021-06-17,03:03:12,NaN,NaN
10797,0_2245,s_49323,c_7954,2021-06-16 05:53:01.627491,June,16,2021,2,6,3,Wednesday,2021-06-16,05:53:01,NaN,NaN
14045,0_2245,s_124474,c_5120,2021-11-24 17:35:59.911427,November,24,2021,3,11,3,Wednesday,2021-11-24,17:35:59,NaN,NaN
17480,0_2245,s_172304,c_4964,2022-02-28 18:08:49.875709,February,28,2022,4,2,1,Monday,2022-02-28,18:08:49,NaN,NaN
21071,0_2245,s_3,c_580,2021-03-01 00:09:29.301897,March,1,2021,1,3,1,Monday,2021-03-01,00:09:29,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322523,0_2245,s_16936,c_4167,2021-04-06 19:59:19.462288,April,6,2021,1,4,2,Tuesday,2021-04-06,19:59:19,NaN,NaN
329226,0_2245,s_13738,c_7790,2021-03-30 23:29:02.347672,March,30,2021,4,3,2,Tuesday,2021-03-30,23:29:02,NaN,NaN
330297,0_2245,s_128815,c_6189,2021-12-03 14:14:40.444177,December,3,2021,1,12,5,Friday,2021-12-03,14:14:40,NaN,NaN
335331,0_2245,s_26624,c_1595,2021-04-27 18:58:47.703374,April,27,2021,4,4,2,Tuesday,2021-04-27,18:58:47,NaN,NaN


In [32]:
tp_missing = tp.loc[tp["id_prod"].str.contains("0_224", case=False)].sort_values(by="id_prod")
print(tp_missing.groupby("id_prod")["price"].sum());

# other method
tp_absent = tp[tp.id_prod.isin(pr.id_prod)==False]["id_prod"].unique()[0];
print(f"\nWe notice that id_prod {tp_absent} doesn't have price nor category recorded,\nso we will have to impute its values by using mean")

id_prod
0_224      200.16
0_2240     297.27
0_2241    1000.09
0_2242     127.16
0_2243     809.10
0_2244     853.74
0_2245       0.00
0_2246     648.90
0_2247      20.93
0_2248    1820.94
0_2249       5.98
Name: price, dtype: float64

We notice that id_prod 0_2245 doesn't have price nor category recorded,
so we will have to impute its values by using mean


In [33]:
# verifying other modalities [monthly]
tp_date_detection = tp.copy()

# sorting relevant values
tp_date_detection.sort_values(by=["month", "day", "categ"], ascending=[True, True, False], inplace=True)
# creation d"un filtre par mois
filt = (tp_date_detection["month"] == "October") & (tp_date_detection["categ"] == 1)

# visulation
print(f"There are no transactions made for Category 1 between October 2 to October 27")
tp_date_detection[filt].groupby(["categ", "month", "day"])[["price"]].sum()


# ---------------------------------------------------------------------------------
# GO BACK HERE!
# enlever octobre de la data set
# trans_prod.drop(trans_prod[trans_prod["month"] == "October"].index, inplace=True)

# verification: tpc.loc[tpc["month"] == "October"] ou
# set(tp.month.unique())

There are no transactions made for Category 1 between October 2 to October 27


price
categ month   day         
1.0   October 1    7003.79
              28   6317.99
              29   6425.18
              30   6753.69
              31   7261.67

#### FIlling missing values of 0_2245

In [34]:
# Fonction permettant de récupérer le premier chiffre d'un identifiant de produit dans nos données:
def get_categ(car):
    return  int(car.split("_")[0])

# Moyenne des prix aux alentours de 0_224
pr_price = pr.loc[pr["id_prod"].str.contains("0_224", case=False)]
pr_price = pd.DataFrame(pr_price)

mean_price = pr_price["price"].mean()
mean_price = round(mean_price, 2)
print(f"Prix moyen des produits 0_224 à 0_2249 est {mean_price}")

# Remplissage des valeurs nuls
tp["categ"] = tp.id_prod.apply(get_categ)
tp["price"].fillna(mean_price, inplace=True)  #tp["price"].replace(np.nan, mean_price, inplace=True)



# GO BACK HERE!
# tp["is_sold"].fillna(True, inplace=True)      #tp["is_sold"].replace(np.nan, True, inplace=True)

Prix moyen des produits 0_224 à 0_2249 est 10.82


In [35]:
# let's verify
print(tp.loc[tp["id_prod"].str.contains("0_224", case=False)].groupby("id_prod")["price"].sum())

tp.loc[tp["id_prod"] == tp_absent][["id_prod", "price", "categ"]]

id_prod
0_224      200.16
0_2240     297.27
0_2241    1000.09
0_2242     127.16
0_2243     809.10
0_2244     853.74
0_2245    1114.46
0_2246     648.90
0_2247      20.93
0_2248    1820.94
0_2249       5.98
Name: price, dtype: float64


,id_prod,price,categ
6231,0_2245,10.82,0
10797,0_2245,10.82,0
14045,0_2245,10.82,0
17480,0_2245,10.82,0
21071,0_2245,10.82,0
...,...,...,...
322523,0_2245,10.82,0
329226,0_2245,10.82,0
330297,0_2245,10.82,0
335331,0_2245,10.82,0


### TPC table -- TP and Customers

In [36]:
# creating the final data set to use for the entire analysis
tpc = pd.merge(tp, cl, on="client_id", how="left")
tpc = tpc.sort_values(by="full_time").reset_index(drop=True); tpc.head(2)

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ,sex,age
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138,March,1,2021,1,3,1,Monday,2021-03-01,00:01:07,11.99,0,f,55
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414,March,1,2021,1,3,1,Monday,2021-03-01,00:02:26,19.37,0,m,62


#### Fixing chronology

In [37]:
# verifying chronology of dates by month
tpc.groupby("month")[["price"]].sum()

,price
month,
April,473132.20
August,479402.57
December,523013.42
February,532859.90
January,523009.41
July,480941.94
June,481333.68
March,479409.41
May,489454.51


In [38]:
# setting [month] column in the right order
months = ["March","April","May","June","July","August",
          "September","October", "November",
          "December","January","February"]

tpc["month"] = pd.CategoricalIndex(tpc["month"], ordered=True, categories=months)
tpc.sort_values(by=["date"])

# let's verify
tpc.groupby("month")[["price"]].sum()

,price
month,
March,479409.41
April,473132.20
May,489454.51
June,481333.68
July,480941.94
August,479402.57
September,502901.12
October,319235.70
November,512998.37


In [39]:
# verifying chronology of dates by day
tpc.groupby("day_name")[["price"]].sum()

,price
day_name,
Friday,838201.67
Monday,843017.46
Saturday,820955.06
Sunday,821362.81
Thursday,830712.63
Tuesday,825164.72
Wednesday,818277.88


In [40]:
# setting [day] column in the right order
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

tpc["day_name"] = pd.CategoricalIndex(tpc["day_name"], ordered=True, categories=days)
tpc.sort_values(by=["date"])

# let's verify
tpc.groupby("day_name")[["price"]].sum()

,price
day_name,
Monday,843017.46
Tuesday,825164.72
Wednesday,818277.88
Thursday,830712.63
Friday,838201.67
Saturday,820955.06
Sunday,821362.81


### Final touches for TPC

#### Supplementary values (Basket size)

In [41]:
# how much books were bought per session
tpc["basket_size"] = tpc["session_id"].value_counts()[tpc["session_id"]].tolist(); tpc.head(2)

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ,sex,age,basket_size
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138,March,1,2021,1,3,1,Monday,2021-03-01,00:01:07,11.99,0,f,55,1
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414,March,1,2021,1,3,1,Monday,2021-03-01,00:02:26,19.37,0,m,62,1


#### Fixing date to datetime

In [42]:
tpc.dtypes

id_prod                object
session_id             object
client_id              object
full_time      datetime64[ns]
month                category
day                     int64
year                    int64
quarterly               int64
_monthnum               int64
_daynum                 int64
day_name             category
date                   object
hour                   object
price                 float64
categ                   int64
sex                    object
age                     int64
basket_size             int64
dtype: object

In [43]:
# date as object to datetime
tpc["date"] = pd.to_datetime(tpc["date"])

# other method to add day and month name (already done in Fixing Date @Transactions)
# sliced_tr["month"] = sliced_tr["date_and_time"].dt.month.apply(month_name)
# sliced_tr["day"] = sliced_tr["date_and_time"].dt.day

#### Binning age in 3 sections

In [44]:
# defining class function
def set_class(a):
    if a<=30: return("student")
    elif a<=50: return("active")
    else: return("senior")

# binning
tpc["client_type"] = tpc["age"].apply(set_class)
# tpc = tpc.reset_index(drop=True)
tpc.head()

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ,sex,age,basket_size,client_type
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138,March,1,2021,1,3,1,Monday,2021-03-01,00:01:07,11.99,0,f,55,1,senior
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414,March,1,2021,1,3,1,Monday,2021-03-01,00:02:26,19.37,0,m,62,1,senior
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413,March,1,2021,1,3,1,Monday,2021-03-01,00:02:38,4.50,0,m,34,4,active
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692,March,1,2021,1,3,1,Monday,2021-03-01,00:04:54,6.55,0,f,33,2,active
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198,March,1,2021,1,3,1,Monday,2021-03-01,00:05:18,16.49,0,f,66,1,senior


#### Arranging columns for the Final Table

In [45]:
tpc.head()

,id_prod,session_id,client_id,full_time,month,day,year,quarterly,_monthnum,_daynum,day_name,date,hour,price,categ,sex,age,basket_size,client_type
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138,March,1,2021,1,3,1,Monday,2021-03-01,00:01:07,11.99,0,f,55,1,senior
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414,March,1,2021,1,3,1,Monday,2021-03-01,00:02:26,19.37,0,m,62,1,senior
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413,March,1,2021,1,3,1,Monday,2021-03-01,00:02:38,4.50,0,m,34,4,active
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692,March,1,2021,1,3,1,Monday,2021-03-01,00:04:54,6.55,0,f,33,2,active
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198,March,1,2021,1,3,1,Monday,2021-03-01,00:05:18,16.49,0,f,66,1,senior


In [46]:
# arranging our main data base:
tpc = tpc.reindex(columns=["id_prod", "categ", "price", "client_id", "sex", "age", "client_type", "session_id", "basket_size", "date", "_daynum", "day_name", "month", "day", "year", "hour", "quarterly", "_monthnum", "full_time"]); tpc.head(2)

,id_prod,categ,price,client_id,sex,age,client_type,session_id,basket_size,date,_daynum,day_name,month,day,year,hour,quarterly,_monthnum,full_time
0,0_1259,0,11.99,c_329,f,55,senior,s_1,1,2021-03-01,1,Monday,March,1,2021,00:01:07,1,3,2021-03-01 00:01:07.843138
1,0_1390,0,19.37,c_664,m,62,senior,s_2,1,2021-03-01,1,Monday,March,1,2021,00:02:26,1,3,2021-03-01 00:02:26.047414


# Exporting final data set

In [47]:
# tpc.to_csv("/content/drive/MyDrive/OC/P4/P4_official_files/P4_importexports/tpc.csv", index = False)
# tp.to_csv("/content/drive/MyDrive/OC/P4/P4_official_files/P4_importexports/tp.csv", index = False)
# tr.to_csv("/content/drive/MyDrive/OC/P4/P4_official_files/P4_importexports/tr.csv", index = False)
# pr.to_csv("/content/drive/MyDrive/OC/P4/P4_official_files/P4_importexports/pr.csv", index = False)
# cl.to_csv("/content/drive/MyDrive/OC/P4/P4_official_files/P4_importexports/cl.csv", index = False)